In [1]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import random
import math
import os
import pandas as pd

In [2]:
mapping = pickle.load(open("pickles_data/graph_embedding_mapping.pkl",'rb'))

In [3]:
id_to_name = pickle.load(open("pickles_data/arnetids_to_papername.pkl","rb"))
name_to_id = pickle.load(open("pickles_data/arnetids_to_papername.pkl","rb"))

In [4]:
tags = pickle.load(open("pickles_data/baseline_tags.pkl","rb"))

In [5]:
testset = pickle.load(open("pickles_data/testset_for_graph.pkl","rb"))
trainset = pickle.load(open("pickles_data/trainset_for_graph.pkl","rb"))

In [6]:
for data in trainset :
    data['source_embed'] = mapping[data['acl_paper_arnetid']]
    data['dest_embed'] = mapping[data['arnet_id']]

In [7]:
for data in testset :
    data['source_embed'] = mapping[data['acl_paper_arnetid']]
    data['dest_embed'] = mapping[data['arnet_id']]

In [8]:
weighted_cue = pickle.load(open("pickles_data/weighted_cue_words.pkl", "rb"))
bert_embeddings = pickle.load(open("pickles_data/bert_embeddings_relu.pkl", "rb"))

In [9]:
location_feature = pickle.load(open("pickles_data/location_feature.pkl","rb"))
title_overlap = pickle.load(open("pickles_data/title_overlap.pkl","rb"))
context_count = pickle.load(open("pickles_data/context_count.pkl","rb"))

In [10]:
num_table = pickle.load(open("pickles_data/num_table.pkl","rb"))
popularity = pickle.load(open("pickles_data/popularity_sent.pkl", "rb"))

In [11]:
paper_info = pickle.load(open("pickles/paper_info (1).pickle", "rb"))
fixed_context = pickle.load(open("pickles_data/fixed_context.pkl","rb"))
author_feat = pickle.load(open("pickles/author_level.pickle", "rb"))

In [12]:
count = 0
cbert = 0
count_cc = 0
train_set = []
for data in trainset :
    count+=1
    paper = data['paper_name']
    key = data['acl_paper_id']
    if(key in bert_embeddings) :
        for pap in bert_embeddings[key] :
            if(pap['paper_name']==paper) :
                cbert+=1
                data['bert_embed'] = pap['embedding']

        for pap in paper_info[key] :
            if(pap['paper_name']==paper) :
                try: 
                    data['citation_count'] = int(pap['citation'])
                except :
                    data['citation_count'] = 1
                    count_cc += 1
                try :
                    data['venue_cit'] = int(pap['venue_cit'])
                except :
                    data['venue_cit'] = 20

                info_found = 1
                break


        for pap in location_feature[key]:
            if(pap['paper_name']==paper) :
                data['location_feature'] = pap['location_feature']
                break

        for pap in author_feat[key] :
            if(pap['paper_name']==paper) :
                data['author_hindex'] = pap['hindex']
                data['author_prod'] = pap['productivity']

        for pap in num_table[key]:
            if(pap['paper_name']==paper) :
                data['num_table'] = pap['num_table']
                break

        for pap in title_overlap[key]:
            if(pap['paper_name']==paper) :
                data['title_overlap'] = pap['overlap']
                break

        for pap in popularity[key]:
            if(pap['paper_name']==paper) :
                data['popularity'] = pap['popularity']
                break

        for pap in context_count[key]:
            if(pap['paper_name']==paper) :
                data['context_count'] = pap['context_count']
                break

        for pap in weighted_cue[key]:
            if(pap['paper_name']==paper) :
                max_data = pap['cue_weights_max']
                data['wcue_max'] = []
                for key1 in max_data :
                    data['wcue_max'].append(max_data[key1])
                add_data = pap['cue_weights_add']
                data['wcue_add'] = []
                for key1 in add_data :
                    data['wcue_add'].append(add_data[key1])
                break


        for pap in fixed_context[key]:
            if(pap['paper_name']==paper) :
                data['fixed_context'] = pap['fixed_context']    
                
        train_set.append(data)
            
    


print(cbert)
print(count)

20896
23659


In [13]:
count = 0
cbert = 0
count_cc = 0
test_set = []
for data in testset :
    count+=1
    paper = data['paper_name']
    key = data['acl_paper_id']
    if(key in bert_embeddings) :
        for pap in bert_embeddings[key] :
            if(pap['paper_name']==paper) :
                cbert+=1
                data['bert_embed'] = pap['embedding']

        for pap in paper_info[key] :
            if(pap['paper_name']==paper) :
                try: 
                    data['citation_count'] = int(pap['citation'])
                except :
                    data['citation_count'] = 1
                    count_cc += 1
                try :
                    data['venue_cit'] = int(pap['venue_cit'])
                except :
                    data['venue_cit'] = 20

                info_found = 1
                break


        for pap in location_feature[key]:
            if(pap['paper_name']==paper) :
                data['location_feature'] = pap['location_feature']
                break

        for pap in author_feat[key] :
            if(pap['paper_name']==paper) :
                data['author_hindex'] = pap['hindex']
                data['author_prod'] = pap['productivity']

        for pap in num_table[key]:
            if(pap['paper_name']==paper) :
                data['num_table'] = pap['num_table']
                break

        for pap in title_overlap[key]:
            if(pap['paper_name']==paper) :
                data['title_overlap'] = pap['overlap']
                break

        for pap in popularity[key]:
            if(pap['paper_name']==paper) :
                data['popularity'] = pap['popularity']
                break

        for pap in context_count[key]:
            if(pap['paper_name']==paper) :
                data['context_count'] = pap['context_count']
                break

        for pap in weighted_cue[key]:
            if(pap['paper_name']==paper) :
                max_data = pap['cue_weights_max']
                data['wcue_max'] = []
                for key1 in max_data :
                    data['wcue_max'].append(max_data[key1])
                add_data = pap['cue_weights_add']
                data['wcue_add'] = []
                for key1 in add_data :
                    data['wcue_add'].append(add_data[key1])
                break


        for pap in fixed_context[key]:
            if(pap['paper_name']==paper) :
                data['fixed_context'] = pap['fixed_context']    
                
        test_set.append(data)
            
    


print(cbert)
print(count)

5185
5915


In [14]:
for data in train_set :
    if(data['tag']==1) :
        data['label']='baseline'
    else :
        data['label']='non_baseline'
        
for data in test_set :
    if(data['tag']==1) :
        data['label']='baseline'
    else :
        data['label'] = 'non_baseline'

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
for data in train_set :
    data['dist'] = cosine_similarity(data['source_embed'].reshape(1,-1),data['dest_embed'].reshape(1,-1))[0][0]
    
for data in test_set :
    data['dist'] = cosine_similarity(data['source_embed'].reshape(1,-1),data['dest_embed'].reshape(1,-1))[0][0]

In [17]:
for data in train_set :
    print(data['dist'])
    break

0.661919


In [18]:
def sanity_check(probs):

	sum = 0
	for key, value in probs.items():
		sum+=value
	sum = round(sum, 7)
	return sum==1

In [19]:
def lm_model(trainset, testset):

    word_baseline_frequency = {}
    word_non_baseline_frequency = {}
    baseline_length = 0
    non_baseline_length = 0

    for i in trainset:
        # if len(i['fixed_context'])>1 and i['label']=='baseline':
        # 	continue
        for context in i['fixed_context']:
            if i['label']=='baseline':
                baseline_length+=len(context)
            else:
                non_baseline_length+=len(context)
            for word in context:
                if word not in word_baseline_frequency:
                    word_baseline_frequency[word] = 0
                    word_non_baseline_frequency[word] = 0
                if i['label']=='baseline':
                    word_baseline_frequency[word]+=1
                else:
                    word_non_baseline_frequency[word]+=1

    final_word_baseline_frequency = {}
    final_word_baseline_frequency['<unk>'] = 0
    word_non_baseline_frequency['<unk>'] = 0

    for word in word_baseline_frequency:
        if word_baseline_frequency[word]>5:	
            final_word_baseline_frequency[word] = word_baseline_frequency[word]
        else:
            final_word_baseline_frequency['<unk>'] += word_baseline_frequency[word]
            word_non_baseline_frequency['<unk>'] += word_non_baseline_frequency[word]
            del word_non_baseline_frequency[word]

    word_baseline_frequency = final_word_baseline_frequency

    word_baseline_probability = {}
    word_non_baseline_probability = {}

    for word in word_baseline_frequency:
        word_baseline_probability[word] = (word_baseline_frequency[word]+1)/(baseline_length+len(word_baseline_frequency))
        word_non_baseline_probability[word] = (word_non_baseline_frequency[word]+1)/(non_baseline_length+len(word_non_baseline_frequency))

    # print(word_baseline_probability['<unk>'])
    # print(word_non_baseline_probability['<unk>'])

    assert sanity_check(word_baseline_probability) and sanity_check(word_non_baseline_probability)

    conf_mat1 = np.zeros((2,2))
    conf_mat2 = np.zeros((2,2))

    min_p = 1

    for i in trainset+testset:
        P_b = 0
        P_nb = 0
        maxP_b = 0
        minP_nb = 1
        n_better = 0
        for context in i['fixed_context']:
            
            P_b_con = 1
            P_nb_con = 1
            for word in context:
                if word not in word_baseline_probability:
                    P_b_con*=word_baseline_probability['<unk>']
                    P_nb_con*=word_non_baseline_probability['<unk>']
                else:
                    P_b_con*=word_baseline_probability[word]
                    P_nb_con*=word_non_baseline_probability[word]
            if P_b_con>P_nb_con:
                n_better+=1
            P_b+=P_b_con
            P_nb+=P_nb_con
            maxP_b = max(P_b, P_b_con)
            minP_nb = min(P_nb, P_nb_con)
        P_b/=len(i['fixed_context'])
        P_nb/=len(i['fixed_context'])

        # if P_b>P_nb:
        # 	print('yes')

        # i['P_b'] = P_b
        # i['P_nb'] = P_nb
#         n_better = n_better/len(i['fixed_context'])
        min_p = min(min_p, P_b, P_nb)
#         i['lmp'] = [P_b, P_nb, n_better]
        i['lmp'] = [P_b, P_nb, maxP_b, minP_nb, P_b/P_nb, maxP_b/minP_nb, n_better]

    for i in trainset:	
        # if len(i['fixed_context'])>1 and i['label']=='baseline':
        # 	testset.append(i)
        # 	continue
        i['lmp'][0]/=min_p
        i['lmp'][1]/=min_p
        i['lmp'][2]/=min_p
        i['lmp'][3]/=min_p
        if i['lmp'][0]>i['lmp'][1]:
            if i['label']=='baseline':
                conf_mat1[0, 0]+=1
            else:
                conf_mat1[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat1[1, 0]+=1
            else:
                conf_mat1[1, 1]+=1
        if i['lmp'][2]>i['lmp'][3]:
            if i['label']=='baseline':
                conf_mat2[0, 0]+=1
            else:
                conf_mat2[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat2[1, 0]+=1
            else:
                conf_mat2[1, 1]+=1

    # print(conf_mat1)
    prec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[0,1])
    rec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat1 = np.zeros((2,2))

    # print(conf_mat2)
    prec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[0,1])
    rec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
#     conf_mat2 = np.zeros((2,2))

    for i in testset:
        i['lmp'][0]/=min_p
        i['lmp'][1]/=min_p
        i['lmp'][2]/=min_p
        i['lmp'][3]/=min_p
        if i['lmp'][0]>i['lmp'][1]:
            if i['label']=='baseline':
                conf_mat1[0, 0]+=1
            else:
                conf_mat1[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat1[1, 0]+=1
            else:
                conf_mat1[1, 1]+=1
        if i['lmp'][2]>i['lmp'][3]:
            if i['label']=='baseline':
                conf_mat2[0, 0]+=1
            else:
                conf_mat2[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat2[1, 0]+=1
            else:
                conf_mat2[1, 1]+=1

    # print(conf_mat1)
    prec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[0,1])
    rec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat1 = np.zeros((2,2))

    # print(conf_mat2)
    prec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[0,1])
    rec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat2 = np.zeros((2,2))

In [20]:
lm_model(train_set,test_set)

0.28828139935002867 0.6651962946625496 0.4022405974926647
0.25510485651214126 0.8156153506837229 0.38864950078822913
0.27034120734908135 0.577570093457944 0.36829558998808104
0.2520735261152208 0.8026409707351891 0.3836574547935859


In [21]:
dataset = train_set+test_set

In [22]:
valbase = 0
valnonbase = 0
countbase = 0
countnonbase = 0
for data in train_set :
    if(data['label']=='baseline') :
        valbase+=data['dist']
        countbase+=1
    else :
        valnonbase+=data['dist']
        countnonbase+=1
        
print(valbase/countbase)
print(valnonbase/countnonbase)

valbase = 0
valnonbase = 0
countbase = 0
countnonbase = 0
for data in test_set :
    if(data['label']=='baseline') :
        valbase+=data['dist']
        countbase+=1
    else :
        valnonbase+=data['dist']
        countnonbase+=1
        
print(valbase/countbase)
print(valnonbase/countnonbase)

0.7509819480636123
0.6846241500895507
0.30856580436926023
0.22246808549935848


In [23]:
values = []
output = []
for data in dataset :
    try :
        ar = []
        count = 0
        ar.append(data['dist'])
        ar.append(data['context_count'])   
        ar.append(data['title_overlap'])
        ar.append(data['author_hindex'])
        ar.append(data['author_prod'])
        ar.append(data['popularity'])
        ar.append(data['citation_count'])
        ar.append(data['num_table'])
        ar.extend(data['location_feature'])
        ar.extend(data['lmp'])
        ar.extend(data['wcue_max'])
        ar.extend(data['bert_embed'])
#         ar.extend(data['source_embed'])

        values.append(ar)
        if(data['label']=='baseline'):
            output.append(1)
        else :
            output.append(0)
            
    except :
        pass

In [24]:
values = np.array(values)
output = np.array(output)

In [25]:
print(values.shape)

(25968, 321)


In [26]:
scaler = MinMaxScaler((0,1))
values = scaler.fit_transform(values)

In [30]:
values, output = shuffle(values,output)

In [31]:
def split(data, output):
    n = len(data)
    last = int(0.8*n)
    train_data = data[:last]
    train_output = output[:last]
    test_data = data[last:]
    test_output = output[last:]
    return train_data, test_data, train_output, test_output

In [32]:
train_data, test_data, train_output, test_output = split(values, output)

In [33]:
def shuffle(train_data, train_output) :
    baselines = []
    non_baselines = []
    for i in range(len(train_output)) :
        if(train_output[i]==1) :
            baselines.append(train_data[i])
        else :
            non_baselines.append(train_data[i])
    
    n = len(baselines)
    ar = np.random.choice(len(non_baselines), len(baselines))
    nb_ar = []
    for x in ar :
        nb_ar.append(non_baselines[x])
        
    data = []
    data.extend(nb_ar)
    output = []
    for i in range(len(data)):
        output.append(0)
    
    data.extend(baselines)
    for i in range(n) :
        output.append(1)
        
    total_data = []
    for i in range(len(data)):
        ar = []
        ar.append(data[i])
        ar.append(output[i])
        total_data.append(ar)
        
    total_data = np.array(total_data)
    np.random.shuffle(total_data)
    
    data = []
    output = []
    for ar in total_data :
        data.append(ar[0])
        output.append(ar[1])
        
    data = np.array(data)
    output = np.array(output)
    
    return data, output

In [ ]:
# train_data, train_output = shuffle(train_data, train_output)

In [ ]:
params = {'C':[0.1, 0.5, 1, 2], 'fit_intercept':[True, False]}
modelin = LogisticRegression(solver='lbfgs', max_iter=1e4, n_jobs=5, random_state=1, warm_start=False)
model = GridSearchCV(modelin, params, cv=2, n_jobs=5)
# model = LogisticRegression(solver='lbfgs',max_iter=1e4, n_jobs=5, random_state=1, warm_start=False, C=1, fit_intercept=True)
clf = model.fit(train_data, train_output)
predict_test = clf.predict(test_data)
print(classification_report(test_output, predict_test))
predictions = clf.predict(train_data)
print(classification_report(train_output, predictions))
print(model.best_params_)


import copy
normal = copy.deepcopy(clf.best_estimator_.coef_[0])
coef = clf.best_estimator_.coef_[0][:64]
print(coef)
ar = clf.best_estimator_.coef_[0]
ar.sort()
# print(ar)

for i in range(15) :
    print(ar[len(ar)-1-i], list(normal).index(ar[len(ar)-1-i]))
# print(clf.best_estimator_.coef_[0])

In [ ]:
params = {'C':[0.1, 0.5, 1, 2]}
rbf_svc = svm.SVC(kernel='rbf', gamma='scale', max_iter=1e4, C=1 )
model = GridSearchCV(rbf_svc, params, cv=2, n_jobs=5)
clf = model.fit(train_data, train_output)
predict_test = clf.predict(test_data)
print(classification_report(test_output, predict_test))